In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## block

In [2]:

class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self, input_size: int, theta_size: int, horizon: int, n_neurons: int, n_layers: int, **kwargs):
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs):
    x = inputs
    for layer in self.hidden:
      x = layer(x)
    theta = self.theta_layer(x)
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [3]:
HORIZON = 1
WINDOW_SIZE = 7

In [4]:

df = pd.read_csv("/content/Tomato.csv",
                 parse_dates=["Date"],
                 index_col=["Date"])
df.head()

,Unit,Minimum,Maximum,Average,Market
Date,,,,,
2013-06-16,Kg,26,32,29.0,Tomato
2013-06-17,Kg,20,25,22.5,Tomato
2013-06-18,Kg,22,26,24.0,Tomato
2013-06-19,Kg,24,28,26.0,Tomato
2013-06-20,Kg,22,26,24.0,Tomato


In [5]:
df.tail()

,Unit,Minimum,Maximum,Average,Market
Date,,,,,
2021-05-09,Kg,10,15,12.5,Tomato
2021-05-10,Kg,10,15,12.5,Tomato
2021-05-11,Kg,10,15,12.5,Tomato
2021-05-12,Kg,30,35,32.5,Tomato
2021-05-13,Kg,35,40,37.5,Tomato


In [8]:
tomato_prices = pd.DataFrame(df["Average"]).rename(columns={"Average": "Price"})
tomato_prices.head()

,Price
Date,
2013-06-16,29.0
2013-06-17,22.5
2013-06-18,24.0
2013-06-19,26.0
2013-06-20,24.0


In [9]:
tomato_prices_nbeats.describe()

,Price
count,2741.000000
mean,38.185516
std,16.970949
min,10.000000
25%,25.000000
50%,35.000000
75%,47.500000
max,117.500000


In [25]:
df.dtypes

Currency                object
Closing Price (USD)    float64
24h Open (USD)         float64
24h High (USD)         float64
24h Low (USD)          float64
dtype: object

In [10]:
tomato_prices_nbeats = tomato_prices.copy()
for i in range(WINDOW_SIZE):
  tomato_prices_nbeats[f"Price+{i+1}"] = tomato_prices_nbeats["Price"].shift(periods=i+1)
tomato_prices_nbeats.dropna().head()

,Price,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6,Price+7
Date,,,,,,,,
2013-06-26,22.5,27.5,17.5,24.0,26.0,24.0,22.5,29.0
2013-06-27,21.0,22.5,27.5,17.5,24.0,26.0,24.0,22.5
2013-06-28,26.0,21.0,22.5,27.5,17.5,24.0,26.0,24.0
2013-06-30,20.0,26.0,21.0,22.5,27.5,17.5,24.0,26.0
2013-07-01,17.5,20.0,26.0,21.0,22.5,27.5,17.5,24.0


In [12]:
X = tomato_prices_nbeats.dropna().drop("Price", axis=1)
y = tomato_prices_nbeats.dropna()["Price"]



split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)

(2187, 2187, 547, 547)

In [13]:

train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(y_train)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

BATCH_SIZE = 1024
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>)

In [14]:

N_EPOCHS = 5000
N_NEURONS = 512
N_LAYERS = 4
N_STACKS = 30

INPUT_SIZE = WINDOW_SIZE * HORIZON
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(7, 8)

In [15]:
from tensorflow.keras import layers
tf.random.set_seed(42)

nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS)

stack_input = layers.Input(shape=(INPUT_SIZE))
backcast, forecast = nbeats_block_layer(stack_input)
residuals = layers.subtract([stack_input, backcast])

for i, _ in enumerate(range(N_STACKS-1)):
  backcast, block_forecast = NBeatsBlock(
      input_size=INPUT_SIZE,
      theta_size=THETA_SIZE,
      horizon=HORIZON,
      n_neurons=N_NEURONS,
      n_layers=N_LAYERS
  )(residuals)

  residuals = layers.subtract([residuals, backcast])
  forecast = layers.add([forecast, block_forecast])

model = tf.keras.Model(inputs=stack_input,
                         outputs=forecast)


model.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["mae", "mse"])

model.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])

Epoch 1/5000
3/3 [==============================] - 55s 894ms/step - loss: 562.9211 - mae: 562.9211 - mse: 723072.0625 - val_loss: 72.3518 - val_mae: 72.3518 - val_mse: 6913.8774 - lr: 0.0010
Epoch 2/5000
3/3 [==============================] - 0s 142ms/step - loss: 44.7851 - mae: 44.7851 - mse: 3910.7837 - val_loss: 41.0028 - val_mae: 41.0028 - val_mse: 2244.9827 - lr: 0.0010
Epoch 3/5000
3/3 [==============================] - 0s 141ms/step - loss: 28.4280 - mae: 28.4280 - mse: 1169.6272 - val_loss: 11.8099 - val_mae: 11.8099 - val_mse: 229.2505 - lr: 0.0010
Epoch 4/5000
3/3 [==============================] - 0s 134ms/step - loss: 8.1663 - mae: 8.1663 - mse: 119.9875 - val_loss: 7.1055 - val_mae: 7.1055 - val_mse: 89.5723 - lr: 0.0010
Epoch 5/5000
3/3 [==============================] - 0s 141ms/step - loss: 6.6095 - mae: 6.6095 - mse: 71.0595 - val_loss: 6.2582 - val_mae: 6.2582 - val_mse: 77.4740 - lr: 0.0010
Epoch 6/5000
3/3 [==============================] - 0s 133ms/step - loss: 5.

In [16]:

model.evaluate(test_dataset)

1/1 [==============================] - 0s 81ms/step - loss: 4.4840 - mae: 4.4840 - mse: 44.9770


[4.48399019241333, 4.48399019241333, 44.97698211669922]

In [17]:
def make_preds(model, input_data):

  forecast = model.predict(input_data)
  return tf.squeeze(forecast)

In [18]:
temp = make_preds(model , test_dataset)

1/1 [==============================] - 1s 1s/step


In [19]:
test_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

In [20]:
temp[-10:]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([21.837675, 14.84544 , 16.61057 , 13.963288, 16.219543, 14.645857,
       12.418736, 12.582252, 12.527301, 29.564888], dtype=float32)>

In [21]:
model.save('prices.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
